[그림 4.3] 2010년부터 2016년까지 KOSPI 200 내에서의 KOSPI 200 지수, 2배 레버리지, 3배 레버지지의  Volatility Drag 효과를 확인

In [4]:
import pandas as pd
import numpy as np
import pandas_datareader.data as web
from datetime import datetime
import matplotlib.pyplot as plt
import yfinance as yf

In [5]:
start = datetime(2010, 1, 1)
end = datetime(2016, 12, 31)
ticker = "^KS200"  # KOSPI 200 지수

stock_data = yf.download(ticker, start=start, end=end)
stock_data = stock_data.astype('float')
stock_data.info()

[*********************100%%**********************]  1 of 1 completed

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1695 entries, 2010-01-04 to 2016-12-29
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       1695 non-null   float64
 1   High       1695 non-null   float64
 2   Low        1695 non-null   float64
 3   Close      1695 non-null   float64
 4   Adj Close  1695 non-null   float64
 5   Volume     1695 non-null   float64
dtypes: float64(6)
memory usage: 92.7 KB


In [ ]:
# 로그 수익률 계산
stock_data['Log_Return'] = np.log(stock_data['Adj Close'] / stock_data['Adj Close'].shift(1))

# 레버리지 수익률 계산
stock_data['2x_Log_Return'] = 2 * stock_data['Log_Return']
stock_data['3x_Log_Return'] = 3 * stock_data['Log_Return']

In [6]:

# 로그 수익률 계산
kospi200['Log_Return'] = np.log(kospi200['Adj Close'] / kospi200['Adj Close'].shift(1))

# 레버리지 수익률 계산
kospi200['2x_Log_Return'] = 2 * kospi200['Log_Return']
kospi200['3x_Log_Return'] = 3 * kospi200['Log_Return']

# 평균 및 표준편차 계산
mean_return = kospi200['Log_Return'].mean()
std_dev = kospi200['Log_Return'].std()
mean_return_2x = kospi200['2x_Log_Return'].mean()
std_dev_2x = kospi200['2x_Log_Return'].std()
mean_return_3x = kospi200['3x_Log_Return'].mean()
std_dev_3x = kospi200['3x_Log_Return'].std()

# 기하 평균 수익률 계산
geo_mean_return = mean_return - (std_dev ** 2) / 2
geo_mean_return_2x = mean_return_2x - (std_dev_2x ** 2) / 2
geo_mean_return_3x = mean_return_3x - (std_dev_3x ** 2) / 2

# Volatility Drag 계산
volatility_drag = mean_return - geo_mean_return
volatility_drag_2x = mean_return_2x - geo_mean_return_2x
volatility_drag_3x = mean_return_3x - geo_mean_return_3x

# 결과 출력
print(f"KOSPI 200 Mean Return: {mean_return * 252:.4f}")
print(f"KOSPI 200 Standard Deviation: {std_dev * np.sqrt(252):.4f}")
print(f"KOSPI 200 Geometric Mean Return: {geo_mean_return * 252:.4f}")
print(f"KOSPI 200 Volatility Drag: {volatility_drag * 252:.4f}")

print(f"2x Leverage Mean Return: {mean_return_2x * 252:.4f}")
print(f"2x Leverage Standard Deviation: {std_dev_2x * np.sqrt(252):.4f}")
print(f"2x Leverage Geometric Mean Return: {geo_mean_return_2x * 252:.4f}")
print(f"2x Leverage Volatility Drag: {volatility_drag_2x * 252:.4f}")

print(f"3x Leverage Mean Return: {mean_return_3x * 252:.4f}")
print(f"3x Leverage Standard Deviation: {std_dev_3x * np.sqrt(252)::.4f}")
print(f"3x Leverage Geometric Mean Return: {geo_mean_return_3x * 252:.4f}")
print(f"3x Leverage Volatility Drag: {volatility_drag_3x * 252:.4f}")

# 그래프 생성
plt.figure(figsize=(14, 10))

# KOSPI 200 지수 종가
plt.subplot(4, 1, 1)
plt.plot(kospi200.index, kospi200['Adj Close'], label='KOSPI 200 Adj Close')
plt.title('KOSPI 200 Adjusted Close Price')
plt.legend()

# 일별 로그 수익률
plt.subplot(4, 1, 2)
plt.plot(kospi200.index, kospi200['Log_Return'], label='KOSPI 200 Log Return')
plt.plot(kospi200.index, kospi200['2x_Log_Return'], label='2x Leverage Log Return', alpha=0.7)
plt.plot(kospi200.index, kospi200['3x_Log_Return'], label='3x Leverage Log Return', alpha=0.5)
plt.title('KOSPI 200 Daily Log Return and Leveraged Returns')
plt.legend()

# 기하 평균 수익률과 산술 평균 수익률 비교
labels = ['KOSPI 200', '2x Leverage', '3x Leverage']
mean_returns = [mean_return * 252, mean_return_2x * 252, mean_return_3x * 252]
geo_mean_returns = [geo_mean_return * 252, geo_mean_return_2x * 252, geo_mean_return_3x * 252]
volatility_drags = [volatility_drag * 252, volatility_drag_2x * 252, volatility_drag_3x * 252]

x = np.arange(len(labels))

plt.subplot(4, 1, 3)
plt.bar(x - 0.2, mean_returns, width=0.2, label='Mean Return', color='blue')
plt.bar(x, geo_mean_returns, width=0.2, label='Geometric Mean Return', color='green')
plt.bar(x + 0.2, volatility_drags, width=0.2, label='Volatility Drag', color='red')
plt.xticks(x, labels)
plt.title('Mean Return vs Geometric Mean Return vs Volatility Drag')
plt.legend()

plt.tight_layout()
plt.show()


NameError: name 'kospi200' is not defined

[그림 4.4] 변동성 포트폴리오 누적 수익률

In [ ]:
import pandas as pd
import numpy as np
import pandas_datareader.data as web
from datetime import datetime
import matplotlib.pyplot as plt
from scipy import stats

# Yahoo Finance에서 KOSPI 구성 종목 데이터를 다운로드합니다.
# 실제 KOSPI 종목 데이터를 사용해야 하나 예시로 AAPL, MSFT, GOOG, AMZN, FB를 사용하겠습니다.
tickers = ['AAPL', 'MSFT', 'GOOG', 'AMZN', 'FB']  # 예시 종목
start = datetime(2000, 1, 1)
end = datetime(2016, 12, 31)

# 종목별 데이터를 다운로드합니다.
data = {}
for ticker in tickers:
    data[ticker] = web.DataReader(ticker, 'yahoo', start, end)['Adj Close']

# 데이터프레임으로 변환합니다.
df = pd.DataFrame(data)

# 월별 종가 데이터로 리샘플링합니다.
monthly_df = df.resample('M').ffill()

# 월별 수익률을 계산합니다.
monthly_returns = monthly_df.pct_change().dropna()

# 각 종목이 60개월(5년) 이상 데이터가 있는지 확인합니다.
def filter_stocks(monthly_df, threshold=60):
    valid_stocks = []
    for ticker in monthly_df.columns:
        if monthly_df[ticker].first_valid_index() is not None:
            valid_months = monthly_df[ticker].loc[monthly_df[ticker].first_valid_index():].count()
            if valid_months >= threshold:
                valid_stocks.append(ticker)
    return valid_stocks

valid_stocks = filter_stocks(monthly_df)

# 유효한 종목만으로 데이터프레임 필터링
monthly_df = monthly_df[valid_stocks]
monthly_returns = monthly_returns[valid_stocks]

# 월별 수익률의 60개월 이동 표준편차를 계산합니다.
rolling_std_dev = monthly_returns.rolling(window=60).std()

# 매월 포트폴리오를 5개로 나눕니다.
def assign_portfolios(rolling_std_dev):
    portfolios = {}
    for date, row in rolling_std_dev.iterrows():
        sorted_row = row.dropna().sort_values()
        num_stocks = len(sorted_row)
        portfolio_size = num_stocks // 5
        portfolios[date] = {
            'P1': sorted_row.iloc[:portfolio_size].index.tolist(),
            'P2': sorted_row.iloc[portfolio_size:2*portfolio_size].index.tolist(),
            'P3': sorted_row.iloc[2*portfolio_size:3*portfolio_size].index.tolist(),
            'P4': sorted_row.iloc[3*portfolio_size:4*portfolio_size].index.tolist(),
            'P5': sorted_row.iloc[4*portfolio_size:].index.tolist(),
        }
    return portfolios

portfolios = assign_portfolios(rolling_std_dev)

# 분기별 리밸런싱
def rebalance_portfolios(portfolios):
    rebalanced_portfolios = {}
    for i, date in enumerate(sorted(portfolios.keys())):
        if i % 3 == 0:  # 분기별 리밸런싱
            rebalanced_portfolios[date] = portfolios[date]
    return rebalanced_portfolios

rebalanced_portfolios = rebalance_portfolios(portfolios)

# 포트폴리오 성과 계산
portfolio_returns = {f'P{i+1}': [] for i in range(5)}
dates = []

for date in sorted(rebalanced_portfolios.keys()):
    next_date = date + pd.DateOffset(months=3)
    if next_date in monthly_returns.index:
        for p in range(1, 6):
            portfolio = rebalanced_portfolios[date][f'P{p}']
            if portfolio:
                portfolio_return = monthly_returns.loc[date:next_date, portfolio].mean(axis=1).add(1).prod() - 1
                portfolio_returns[f'P{p}'].append(portfolio_return)
        dates.append(next_date)

# 데이터프레임으로 변환
portfolio_returns_df = pd.DataFrame(portfolio_returns, index=dates)

# Fama-French 데이터 다운로드 및 병합
ff_data = web.DataReader('F-F_Research_Data_Factors', 'famafrench', start, end)[0]
ff_data.index = pd.to_datetime(ff_data.index, format='%Y%m')
ff_data = ff_data.resample('M').last() / 100  # 월별 데이터로 변환 및 퍼센트 단위로 변환

# 포트폴리오 수익률과 Fama-French 요인 병합
merged_data = portfolio_returns_df.join(ff_data, how='inner')

# 포트폴리오 성과 지표 계산 함수
def calculate_performance_metrics(portfolio_returns, ff_factors):
    metrics = {}
    annual_returns_arith = portfolio_returns.mean() * 12
    annual_returns_geom = ((1 + portfolio_returns).prod() ** (12 / len(portfolio_returns))) - 1
    annual_volatility = portfolio_returns.std() * np.sqrt(12)
    sharpe_ratio = annual_returns_arith / annual_volatility
    win_rate = (portfolio_returns > 0).mean()
    max_drawdown = (portfolio_returns.cumsum().cummax() - portfolio_returns.cumsum()).max()
    
    # Alpha, Beta, SMB, HML 계산
    alphas, betas, smbs, hmls = {}, {}, {}, {}
    for p in portfolio_returns.columns:
        X = ff_factors[['Mkt-RF', 'SMB', 'HML']]
        y = portfolio_returns[p] - ff_factors['RF']
        reg_result = np.linalg.lstsq(X, y, rcond=None)
        alphas[p], betas[p], smbs[p], hmls[p] = reg_result[0]
    
    metrics['Annual Return (Arithmetic)'] = annual_returns_arith
    metrics['Annual Return (Geometric)'] = annual_returns_geom
    metrics['Annual Volatility'] = annual_volatility
    metrics['Sharpe Ratio'] = sharpe_ratio
    metrics['Win Rate'] = win_rate
    metrics['Max Drawdown'] = max_drawdown
    metrics['Alpha'] = alphas
    metrics['Beta'] = betas
    metrics['SMB'] = smbs
    metrics['HML'] = hmls
    
    return pd.DataFrame(metrics)

# 각 포트폴리오의 성과 지표 계산
performance_metrics = calculate_performance_metrics(merged_data[portfolio_returns_df.columns], merged_data[['Mkt-RF', 'SMB', 'HML', 'RF']])

# 결과 출력
print(performance_metrics)

# 그래프 생성
plt.figure(figsize=(14, 7))

for p in range(1, 6):
    plt.plot(portfolio_returns_df.index, portfolio_returns_df[f'P{p}'].cumprod(), label=f'P{p}')

plt.title('Portfolio Performance')
plt.xlabel('Date')
plt.ylabel('Cumulative Return')
plt.legend()
plt.show()
